In [3]:
import requests
import os
city = 'nepalgunj'
api_Key = os.getenv('Api_Key')
api_url = 'https://api.api-ninjas.com/v1/airquality?city={}'.format(city)
response = requests.get(api_url, headers={'X-Api-Key': api_Key})
if response.status_code == requests.codes.ok:
    print(response.text)
else:
    print("Error:", response.status_code, response.text)


{"CO": {"concentration": 707.63, "aqi": 8}, "NO2": {"concentration": 2.51, "aqi": 3}, "O3": {"concentration": 43.27, "aqi": 36}, "SO2": {"concentration": 0.16, "aqi": 0}, "PM2.5": {"concentration": 6.94, "aqi": 22}, "PM10": {"concentration": 7.37, "aqi": 6}, "overall_aqi": 36}


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Assume we have a CSV file with all the data
def load_and_preprocess_data(file_path):
    # Load the data
    df = pd.read_csv(file_path)
    
    # Convert date to datetime and extract relevant features
    df['Date'] = pd.to_datetime(df['Date'])
    df['DayOfWeek'] = df['Date'].dt.dayofweek
    df['Month'] = df['Date'].dt.month
    
    # One-hot encode categorical variables (like weather conditions)
    df = pd.get_dummies(df, columns=['WeatherCondition'])
    
    return df

def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

def build_model(input_shape):
    model = Sequential([
        LSTM(64, activation='relu', input_shape=input_shape, return_sequences=True),
        Dropout(0.2),
        LSTM(32, activation='relu'),
        Dropout(0.2),
        Dense(6)  # Predict 6 air quality parameters
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

# Load and preprocess the data
df = load_and_preprocess_data('air_quality_data.csv')

# Select features for the model
features = ['PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'O3', 
            'VehicleCount', 'AverageSpeed', 'IndustrialEmission', 'OperationalHours',
            'DayOfWeek', 'Month'] + [col for col in df.columns if col.startswith('WeatherCondition_')]

# Normalize the data
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[features]), columns=features)

# Create sequences
sequence_length = 7  # Use one week of data to predict the next
X, y = create_sequences(df_scaled.values, sequence_length)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and train the model
model = build_model((X.shape[1], X.shape[2]))
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Function to predict next week's air quality
def predict_next_week(model, last_week_data):
    # Ensure last_week_data is scaled and in the correct shape
    prediction = model.predict(np.array([last_week_data]))
    # Inverse transform the prediction to get actual values
    return scaler.inverse_transform(prediction)[0][:6]  # Return only air quality parameters

# Example usage:
# last_week_data = df_scaled.iloc[-7:].values
# next_week_prediction = predict_next_week(model, last_week_data)
# print("Predicted air quality for next week:", next_week_prediction)